<div style="border:solid Chocolate 2px; padding: 40px">

Привет, меня зовут Александр Куимов. Я буду ревьюером твоего проекта. Ты можешь обращаться ко мне на "ты"😏 Надеюсь, тебя также не смутит, если я буду обращаться к тебе на "ты", но если это неудобно, обязательно скажи об этом!

Пожалуйста, не удаляй мои комментарии, они будут особенно полезны для нашей работы в случае повторной проверки проекта. 

Ты также можешь реагировать на мои комментарии своими по шаблону, показанному чуть ниже. Это нужно, чтобы не создавалась путаница😉

Ты можешь найти мои комментарии, обозначенные <font color='green'>зеленым</font>, <font color='gold'>желтым</font> и <font color='red'>красным</font> цветами, например:

<br/>

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> В случае, если решение на отдельном шаге является полностью правильным.
</div>

<br/>

<div class="alert alert-warning">
    <h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> В случае, когда решение на отдельном шаге станет еще лучше, если внести небольшие коррективы.
</div>


<br/>
<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера <a class="tocSkip"></h2>

    
<b>На доработку🤔:</b>
 В случае, когда решение на отдельном шаге требует существенной переработки и внесения правок. Напоминаю, что проект не может быть принят с первого раза, если ревью содержит комментарии, рекомендующие доработать шаги.
</div>
    
    
<br/>    
<div class="alert alert-info">
<h2> Комментарий студента: <a class="tocSkip"> </h2>

<b>👋:</b> В такой цветовой ячейке я прошу тебя оставлять свои комментарии. Если исправляешь проект на второй итерации и выше, не забывай пожалуйста указывать номер итерации, например, "Комментарий студента v.2".
</div> 

<br/>    
    
Увидев у тебя неточность, в первый раз я лишь укажу на ее наличие и дам тебе возможность самому найти и исправить ее. На реальной работе твой руководитель будет поступать также, и я пытаюсь подготовить тебя именно к работе датасаентистом. Но если ты пока не справишься с такой задачей - при следующей проверке я дам более точную подсказку!🤓

# Анализ Модели

## Исследовательский анализ данных

In [3]:
import os
import pandas as pd
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib import rcParams

In [8]:
cl = os.path.isfile('/datasets/faces/labels.csv')
# try:
#     cl = os.path.isfile('/datasets/faces/labels.csv')
#     print('in cloud', cl)
# except FileNotFoundError:
#     lo = os.path.isfile('./datasets/labels.csv')
#     print('local', lo)
    

In [9]:
print('in cloud', cl)

in cloud False


In [2]:
!kaggle datasets download -d abhikjha/appa-real-face-cropped

/bin/bash: kaggle: command not found


In [3]:
!unzip appa-real-face-cropped.zip -d datasets

/bin/bash: unzip: command not found


In [4]:
%config InlineBackend.figure_formats = ['svg']

In [5]:
# style MATPLOTLIBRC
custom_params = {
                'figure.figsize': (10, 6),
                'figure.facecolor': '#232425',
                'figure.dpi': 240,

                'legend.frameon': False,
                'legend.borderpad': 1.4,
                'legend.labelspacing': 0.7,
                'legend.handlelength': 0.7,
                'legend.handleheight': 0.7,

                'axes.facecolor': '#232425',
                'axes.labelcolor': '#EEEEEE',
                'axes.labelpad': 17,
                'axes.spines.left': False,
                'axes.spines.bottom': False,
                'axes.spines.right': False,
                'axes.spines.top': False,
                'axes.grid': False,

                'contour.linewidth': 0.0,

                'xtick.color': '#AAAAAA',
                'ytick.color': '#AAAAAA',
                'xtick.bottom': True,
                'xtick.top': False,
                'ytick.left': True,
                'ytick.right': False,
    
                "lines.color": '#EEEEEE',

                'text.color': '#EEEEEE',
    
                'font.family': 'sans-serif',
            }

In [6]:
PATH_TO = './datasets/'
PATH_TO_CSV = PATH_TO + 'labels.csv'
PATH_TO_IMG = PATH_TO + 'final_files/final_files'
dpi_k = custom_params['figure.dpi'] / rcParams['figure.dpi']
px = 1/custom_params['figure.dpi']

In [7]:
def to_px(size_px):
    px_density = int(
        round(
            size_px * px * dpi_k
        )
    )
    return px_density

In [8]:
def plot_hist(data, title='Histogram', custom_params=custom_params):
    with mpl.rc_context(custom_params):
        plt.hist(
            data,
            bins=100 #len(data.unique())
        )
        plt.show()

In [9]:
def plot_16_img(file_names_to_plot):
    fig = plt.figure(
            figsize=(to_px(560), to_px(560)),
            tight_layout=True,
        )
    i=0
    for name in file_names_to_plot:
        i += 1
        fig.add_subplot(4, 4, i)

        img = Image.open(PATH_TO_IMG + '/' + name)
        plt.imshow(np.array(img))

        plt.xticks([])
        plt.yticks([])
#         plt.tight_layout()

In [10]:
labels_df = pd.read_csv(PATH_TO_CSV)
display(labels_df.head())

FileNotFoundError: [Errno 2] No such file or directory: './datasets/labels.csv'

<div style="border:solid Chocolate 2px; padding: 40px">


<h2> Итоговый комментарий ревьюера v.0 <a class="tocSkip"> </h2>    

Некоторые блоки кода не работают. Посмотри, пожалуйста, что пошло не так. Перед отправкой проекта стоит проверять работоспособность кода - это можно сделать, нажав на панели `Jupiter Hub Kernel` и `Restart & Run All`
      
    
Жду от тебя новую версию проекта.    

In [ ]:
display(
    labels_df['real_age'].describe()
)

In [ ]:
 plot_hist(labels_df['real_age'])

In [ ]:
labels_df.groupby(by='real_age').count().sort_values(by='file_name', ascending=False)

Интересно что возросты 25 и 30 самые популярные в наборе данных.  
Распределение приближено к нормальному, но смещено вправо, а в левой части есть подьём гистограммы на значениях около 5 лет.

In [ ]:
plot_16_img(labels_df['file_name'].sample(16))

In [ ]:
quantile_left = labels_df['real_age'].quantile(0.005)
quantile_right = labels_df['real_age'].quantile(0.995)

In [ ]:
print(
    quantile_left,
    quantile_right,
    sep='\n'
)

In [ ]:
faces_outside_quantiles = labels_df.loc[
        labels_df['real_age'] >= quantile_right,
        'file_name'
    ].sample(8)

faces_outside_quantiles = pd.concat(
        [labels_df.loc[
                labels_df['real_age'] <= quantile_left,
                'file_name'
            ].sample(8),
         faces_outside_quantiles
        ],
        axis=0,
        ignore_index=True
    )

In [ ]:
plot_16_img(faces_outside_quantiles)

## Вывод Модели Из GPU

## Анализ Обучения Модели

Похожего результата можно было добиться и за 14 эпох..  
Использовал слой dropout (Dropout), потомучто нашел совет на kaggle, чтобы модель не переобучалась.